In [1]:
import os
import tushare as ts
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

import sys
sys.path.append('..')  # 将父文件夹添加到 Python 路径中，不然找不到下面的包

from utilities.tushare_limit_up import *

以下代码是utilities.tushare_limit_up中generate_limit_up函数的展开

In [3]:
if os.getenv("ENV") != "product":
    # 在开发环境中加载 .env 文件
    load_dotenv(override=True)

# 初始化pro接口
token = os.getenv('TUSHARE_API_KEY')
pro = ts.pro_api(token)

# Generate today's date in "yyyymmdd" format
today_str = datetime.now().strftime("%Y%m%d")
cal_date = getLatestTradeData(pro, today_str)

cal_date

'20240314'

# 以下是getLimitUpData的展开
limit_list_d接口获取到的数据是收盘价等于涨停板的股票
daily中进行筛选获取涨幅大于10%的股票
两者取股票代码的并集

然后对基础的数据进行合并

In [9]:
limit_up_df = pro.limit_list_d(**{ "trade_date": cal_date, "limit_type": "U", }, fields=[ "trade_date", "ts_code", "limit_amount", "fd_amount", "first_time", "last_time", "open_times", "up_stat", "limit_times", "limit" ])
daily_df = pro.daily(**{"trade_date": cal_date}, fields=[ "ts_code", "trade_date", "open", "high", "low", "close", "pre_close", "change", "pct_chg", "vol", "amount" ])

stock_basic_df = pro.stock_basic(**{"list_status": 'L'}, fields=['ts_code','name','industry'])
daily_basic_df = pro.daily_basic(**{"trade_date": cal_date}, fields=['ts_code','trade_date','circ_mv','total_mv','turnover_rate','turnover_rate_f','pe','pb'])


In [ ]:
tenpct_df = daily_df[daily_df['pct_chg'] >= 10]

# 合并tenpct_df和limit_up_df
union_tscode = pd.concat([tenpct_df['ts_code'], limit_up_df['ts_code']]).drop_duplicates()
union_tscode_df = daily_df[daily_df['ts_code'].isin(union_tscode)]
merged_df = pd.merge(union_tscode_df, stock_basic_df, on=['ts_code'], how='left')
merged_df = pd.merge(merged_df, daily_basic_df, on=['ts_code'], how='left')
merged_df = pd.merge(merged_df, limit_up_df, on=['ts_code'], how='outer')

df = merged_df

In [4]:
# 以下是最终代码，直接可调用
df, cal_date = generate_limit_up_df('20240313')

df, cal_date

(      ts_code  name industry  close   pct_chg     amount     circ_mv  \
 52  603261.SH  立航科技       航空  45.84   10.0072   6.241033   12.054671   
 56  603580.SH  艾艾精工       塑料  18.41    9.9761   2.146402   24.056936   
 13  002298.SZ  中电兴发     软件服务   6.36   10.0346   0.501243   40.673719   
 25  002902.SZ  铭普光磁     通信设备  34.43   10.0000  21.767576   51.880130   
 55  603533.SH  掌阅科技      互联网  24.90    9.9823  12.825755  109.285312   
 ..        ...   ...      ...    ...       ...        ...         ...   
 35  300900.SZ  广联航空       航空  24.77   13.1050   4.493151   38.457901   
 37  301285.SZ   鸿日达      元器件  26.66   14.6174   2.590702   20.584135   
 38  301588.SZ   N美新     家居用品  37.39  157.8621   6.608119    8.443597   
 66  688695.SH   N中创     软件服务  48.65  116.8970   6.693719    8.749734   
 67  836395.BJ  朗鸿科技     IT设备  10.21   16.0227   0.698242    2.825961   
 
       total_mv      pb        pe  turnover_rate  turnover_rate_f limit_amount  \
 52   35.901806  4.1119  102.2995       

In [5]:
generate_limit_up_excel(df, cal_date)

已写入路径：./涨停分析-20240313.xlsx
